In [70]:
import torch
import numpy as np


from LSData import LSData
N=100000
region=[-25., 25, -30, 30, -10, 60]
x01 = np.random.uniform(region[0],region[1],(N, 1))
x02 = np.random.uniform(region[2],region[3],(N, 1))
x03 = np.random.uniform(region[4],region[5],(N, 1))

x0=np.hstack([x01, x02, x03])/10
print(x0.shape)
x = torch.tensor(x0, dtype = torch.float)

seed=5
es=np.zeros([3,seed])
ef=np.zeros([3,seed])



def error(data):
    return torch.sqrt(((data**2).mean()).detach())  
    # return data.abs().mean().detach()

for s in range(seed):
    ########## DynGMA
    local = 'outputs/LS_28_seed{}/model_best.pkl'.format(s)
    net = torch.load(local, map_location=torch.device('cpu'))
    
    diff = ((LSData.f(x) - net.vf(x))**2).mean().detach()
    reladiff = torch.sqrt(diff/((LSData.f(x)**2).mean()).detach())    
    
    ef[0, s]=error(LSData.f(x) - net.vf(x))/error(LSData.f(x)) 
 

    NetSigma = net.sigma(x)@net.sigma(x).transpose(dim0=-2, dim1=-1)
    TrueSigma = LSData.g(x)@LSData.g(x).transpose(dim0=-2, dim1=-1)
    sdiff = ((NetSigma - TrueSigma)**2).mean().detach()
    relasdiff = torch.sqrt( sdiff/((TrueSigma**2).mean()).detach())
    
    es[0, s]=error(NetSigma - TrueSigma)/error(TrueSigma) 
    
    
    ######E-M
    local = 'outputs/LSem_28_seed{}/model_best.pkl'.format(s)
    net = torch.load(local, map_location=torch.device('cpu'))
 
    ef[1, s]=error(LSData.f(x) - net.vf(x))/error(LSData.f(x)) 
 

    NetSigma = net.sigma(x)@net.sigma(x).transpose(dim0=-2, dim1=-1)
    TrueSigma = LSData.g(x)@LSData.g(x).transpose(dim0=-2, dim1=-1)
    sdiff = ((NetSigma - TrueSigma)**2).mean().detach()
    relasdiff = torch.sqrt( sdiff/((TrueSigma**2).mean()).detach())
    
    es[1, s]=error(NetSigma - TrueSigma)/error(TrueSigma)
    
    ########## Cub
    local = 'outputs/LScub_28_seed{}/model_best.pkl'.format(s)
    net = torch.load(local, map_location=torch.device('cpu'))
    
    diff = ((LSData.f(x) - net.vf(x))**2).mean().detach()
    reladiff = torch.sqrt(diff/((LSData.f(x)**2).mean()).detach())    
    
    ef[2, s]=error(LSData.f(x) - net.vf(x))/error(LSData.f(x)) 
 

    NetSigma = net.sigma(x)@net.sigma(x).transpose(dim0=-2, dim1=-1)
    TrueSigma = LSData.g(x)@LSData.g(x).transpose(dim0=-2, dim1=-1)
    sdiff = ((NetSigma - TrueSigma)**2).mean().detach()
    relasdiff = torch.sqrt( sdiff/((TrueSigma**2).mean()).detach())
    
    es[2, s]=error(NetSigma - TrueSigma)/error(TrueSigma)   
 

(100000, 3)


In [71]:
print(ef)

[[0.07278875 0.0956483  0.08467264 0.07026868 0.06950027 0.07606915]
 [0.12416982 0.12745562 0.13395989 0.12787673 0.12372219 0.12437012]
 [0.17452079 0.14415167 0.15227385 0.17068934 0.17178693 0.17011039]]


In [72]:
print(es)

[[0.13220727 0.24502853 0.13970622 0.13527371 0.18168548 0.09695784]
 [0.20983623 0.20701794 0.18329178 0.17942153 0.19496793 0.20608576]
 [0.62525916 0.56682831 0.30779764 0.44432259 0.53215939 1.00997579]]


In [73]:
def printvalue(x):
    return format(x, '.2e').replace("e-0", "e-")

print('$e_f$', end=" ")   
for i in range(3):
    print('&', printvalue(ef[i].mean())+'$\pm$'+printvalue(ef[i].std()), end=" ")
print('\\\ $e_{\sigma}$', end=" ") 
for i in range(3):
    print('&', printvalue(es[i].mean())+'$\pm$'+printvalue(es[i].std()), end=" ")
print('\cr')

$e_f$ & 7.82e-2$\pm$9.30e-3 & 1.27e-1$\pm$3.54e-3 & 1.64e-1$\pm$1.14e-2 \\ $e_{\sigma}$ & 1.55e-1$\pm$4.71e-2 & 1.97e-1$\pm$1.19e-2 & 5.81e-1$\pm$2.17e-1 \cr
